In [2]:
#Import necessary libraries
import glob
import os, random
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join
import pandas as pd
import glob
import shutil

In [4]:
#Specify link to train folder
cwd = getcwd()
base_path = cwd + '/train/'

img_path = []
annot_path = []

#Loop through each folder of countries
for  each_folder in sorted(os.listdir(base_path)):
  if each_folder[-5:] != 'pbtxt':
    new_path = base_path + each_folder + "/annotations/" + 'xmls/'
    for files in sorted(os.listdir(new_path)):
        #Save the path to the annotation files
      annot_path.append(new_path + files)


for  each_folder in sorted(os.listdir(base_path)):
  if each_folder[-5:] != 'pbtxt':
    new_path = base_path + each_folder + "/images/"
    for image in sorted(os.listdir(new_path)):
        #Save the path to the image files
     img_path.append(new_path + image)

#Save the path in a DataFrame

image_table = pd.DataFrame(
    {
        "img_path": img_path,
        "annot_path": annot_path
    }
)


In [5]:
clean_index = []
labels = []

#Loop throw each row in the data table
for i, row in image_table.iterrows():
    tree = ET.parse(row['annot_path'])
    root = tree.getroot()
   
    for elem in root:
        for subelem in elem:
            if ('D00' in subelem.text) or ('D10' in subelem.text) or ('D20' in subelem.text) or ('D40' in subelem.text):
                labels.append(subelem.text)
                if i not in clean_index:
                    clean_index.append(i)
#Remove clean data from master table                    
clean_data = image_table.loc[image_table.index.isin(clean_index)]

In [6]:

classes = ['D00', 'D10', 'D20', 'D40'] #Classes we are interested in

def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.jpg'):
        image_list.append(filename)

    return image_list

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)


def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]

    in_file = open(dir_path + '/' + basename_no_ext + '.xml')
    out_file = open(output_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
#         difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls in classes: #Double check data contains interested classes
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w,h), b)
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

In [8]:
# Loop through each clean image

for i in range(len(clean_data)):
    img_dir = clean_data.iloc[i].img_path
    cast = clean_data.iloc[i].annot_path.split('/')[1:-1]
    cast2 = clean_data.iloc[i].annot_path.split('/')[1:-3]
    xml_dir = '/'+"/".join([str(item) for item in cast])
    full_path = '/'+"/".join([str(item) for item in cast2])
    output_path = full_path + '/labels/' 
    clean_img_dest = full_path + '/clean_img/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    if not os.path.exists(clean_img_dest):
        os.makedirs(clean_img_dest)

    shutil.copy(img_dir, clean_img_dest)
    convert_annotation(xml_dir, output_path, img_dir)
# list_file.close()
print("Finished processing all clean images")

Finished processing all clean images


In [9]:
#Copy all Clean images and labels to a single folder
cwd = getcwd()

train_img = cwd + '/final_train/train_imgs/'
label_img = cwd + '/final_train/train_labels/'

if not os.path.exists(train_img):
    os.makedirs(train_img)
    
if not os.path.exists(label_img):
    os.makedirs(label_img)


base_path = cwd + '/train/'

for  each_folder in sorted(os.listdir(base_path)):
  if each_folder[-5:] != 'pbtxt':
    for filename in glob.iglob(f'{base_path + each_folder}/clean_img/*.jpg'):
        shutil.copy(filename, train_img)
    print(f"Successfully copied {each_folder} images to train folder")
    for filename in glob.iglob(f'{base_path + each_folder}/labels/*.txt'):
        shutil.copy(filename, label_img)
    print(f"Successfully copied {each_folder} labels to final_train folder")
    
    
#Since the test files do not have annotations, 
#fetch random 2000 images as validation from train dataset

val_imgs = cwd + '/final_test/val_imgs/'
val_labels = cwd + '/final_test/val_labels/'


if not os.path.exists(val_imgs):
    os.makedirs(val_imgs)
    
if not os.path.exists(val_labels):
    os.makedirs(val_labels)
    
    
for i in range(2000):
    img = random.choice(os.listdir(train_img))
    img_dir = train_img + img
    label_dir = label_img + img.split('.')[0] + ".txt"
    shutil.move(img_dir, val_imgs)
    shutil.move(label_dir, val_labels)

Successfully copied Czech images to train folder
Successfully copied Czech labels to final_train folder
Successfully copied India images to train folder
Successfully copied India labels to final_train folder
Successfully copied Japan images to train folder
Successfully copied Japan labels to final_train folder
